In [ ]:
# 讀取資料
import csv
import numpy as np

max_frame = 26

def load_data(filename):
    X = []
    y = []
    
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            npy_file = row[0]
            label = row[1]
            data = np.load(npy_file)
            if data.shape[0] < max_frame:
                padding_len = max_frame - data.shape[0]
                padding = np.zeros((padding_len, 4))
                data = np.concatenate([data, padding], axis=0)
            X.append(data[:,:3]) # 只保留前三個欄位
            y.append(label)
    
    return np.array(X), np.array(y)


x, y = load_data("/content/drive/MyDrive/Colab/np_label_storage.csv")

idx = np.random.permutation(len(x))

x_shuffled = x[idx]
y_shuffled = y[idx]

for i in range(len(x)):
    print(x_shuffled[i], y_shuffled[i])

串流輸出內容已截斷至最後 5000 行。
 [ 0.23440272  0.47742695 -0.08371525]
 [ 0.18505479  0.56071949 -0.07402192]
 [ 0.20356028  0.54397044 -0.10178014]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]] left wave
[[ 0.16654931  0.57010734 -0.03701096]
 [ 0.14099412  0.54780042 -0.03524853]
 [ 0.12821653  0.53279635 -0.06080372]
 [ 0.10354256  0.4981882  -0.03084246]
 [ 0.07049706  0.44766495  0.01409941]
 [ 0.01586184  0.50884157 -0.03172368]
 [-0.09252739  0.41402754  0.0264364 ]
 [-0.16919294  0.416852    0.05639765]
 [-0.19606995  0.46556655  0.05507583]
 [-0.25511125  0.48487759  0.0594819 ]
 [-0.23660575  0.51035665  0.06212554]
 [-0.31723678  0.46432543  0.07049706]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.   

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

# 定義LSTM模型
model = Sequential()
model.add(LSTM(units=64, input_shape=(26, 3), activation='tanh'))
model.add(Dense(units=2, activation='sigmoid'))

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 將label轉換為one-hot編碼
label_dict = {'left wave': 0, 'right wave': 1}
label = np.array([label_dict[label] for label in y_shuffled])
label = np.eye(2)[label]

# 切分訓練集和驗證集
train_data = x_shuffled[:176]
train_label = label[:176]
val_data = x_shuffled[176:200]
val_label = label[176:200]

# 訓練模型
model.fit(train_data, train_label, validation_data=(val_data, val_label), epochs=15, batch_size=1)


Epoch 1/15
176/176 [==============================] - 3s 7ms/step - loss: 0.4113 - accuracy: 0.7898 - val_loss: 0.0605 - val_accuracy: 1.0000
Epoch 2/15
176/176 [==============================] - 1s 5ms/step - loss: 0.1050 - accuracy: 0.9659 - val_loss: 0.0065 - val_accuracy: 1.0000
Epoch 3/15
176/176 [==============================] - 1s 5ms/step - loss: 0.1880 - accuracy: 0.9375 - val_loss: 0.0135 - val_accuracy: 1.0000
Epoch 4/15
176/176 [==============================] - 1s 6ms/step - loss: 0.0293 - accuracy: 0.9943 - val_loss: 0.1618 - val_accuracy: 0.9167
Epoch 5/15
176/176 [==============================] - 1s 7ms/step - loss: 0.0255 - accuracy: 0.9886 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 6/15
176/176 [==============================] - 1s 6ms/step - loss: 0.0898 - accuracy: 0.9830 - val_loss: 0.0359 - val_accuracy: 1.0000
Epoch 7/15
176/176 [==============================] - 1s 4ms/step - loss: 0.0958 - accuracy: 0.9773 - val_loss: 0.0093 - val_accuracy: 1.0000
Epoch 

In [ ]:
# # 儲存模型權重
# model.save('my_weights')

# 儲存模型權重
model.save_weights('my_weights_1.h5')

In [ ]:
!zip -r 'my_weights.zip' '/content/my_weights'

  adding: content/my_weights/ (stored 0%)
  adding: content/my_weights/assets/ (stored 0%)
  adding: content/my_weights/variables/ (stored 0%)
  adding: content/my_weights/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/my_weights/variables/variables.index (deflated 58%)
  adding: content/my_weights/fingerprint.pb (stored 0%)
  adding: content/my_weights/saved_model.pb (deflated 90%)
  adding: content/my_weights/keras_metadata.pb (deflated 85%)


In [ ]:
from keras.models import load_model
import numpy as np

# # 載入模型
# model = load_model('my_model.h5')

# # 載入新資料，假設新資料的特徵維度為 (220, 26, 3)
# new_data = np.random.rand(220, 26, 3)

test_data = x_shuffled[200:]
test_label = label[200:]

# 進行預測
predictions = model.predict(test_data)

# 將預測結果轉換為類別
predicted_labels = np.argmax(predictions, axis=1)
real_labels = np.argmax(test_label, axis=1)

# 顯示預測結果
print(predictions)
print(real_labels)

1/1 [==============================] - 0s 335ms/step
[[0.9864194  0.26300067]
 [0.81649226 0.22005847]
 [0.00613611 0.88959634]
 [0.9861292  0.2994034 ]
 [0.11622009 0.73027664]
 [0.00605697 0.88963276]
 [0.98644906 0.2624442 ]
 [0.98676795 0.25612906]
 [0.00651428 0.88297457]
 [0.00629072 0.88521945]
 [0.98697704 0.26611683]
 [0.00607995 0.8896321 ]
 [0.00689882 0.8881472 ]
 [0.9867578  0.26615202]
 [0.9864841  0.26034355]
 [0.9852941  0.25693607]
 [0.98697984 0.2622156 ]
 [0.518833   0.22766519]
 [0.00790762 0.88357043]
 [0.00602281 0.8894605 ]]
[0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1]
